In [1]:
import numpy as np
import math
from scipy import sparse

from tqdm import tqdm

## Get data

In [5]:
collection_word = np.load("./lda/model_persistence/all_eths_less_than_15000_1000_its_25_topics/nzwcr.npy")

In [5]:
topic_word = np.load("./lda/model_persistence/all_eths_less_than_15000_1000_its_25_topics/nzw.npy")

In [2]:
c3rLDA_nzwc = np.load("/Users/tingchen/Desktop/Yelp_Cultural_Analysis/lda/model_persistence/all_eths_less_than_15000_1000_its_25_topics/nzwc_ORIGINAL.npy")

In [4]:
reg_assign = np.load("./lda/model_persistence/all_eths_less_than_15000_1000_its_25_topics/reg_assign_mat.npy")
indicies_locs_eth = np.load("./lda/model_persistence/all_eths_less_than_15000_1000_its_25_topics/indicies_locs_eth.npy")

In [9]:
X_val = sparse.load_npz('/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/X_val.npz')
X_test = sparse.load_npz('/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/X_test.npz')
cc_val= np.load("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/y_val.npy")
cc_test= np.load("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/y_test.npy")

total_c = np.concatenate((cc_val, cc_test))
total_X = sparse.vstack([X_val, X_test])

ix = total_X.getnnz(1)>0
total_X = total_X[ix]
total_c= total_c[ix]

In [136]:
reg_assign.shape

(28, 630854)

In [173]:
reg_assign[0]

array([0, 3, 0, ..., 0, 0, 0], dtype=int32)

In [175]:
indicies_locs_eth.shape

(3338332,)

In [177]:
max_reg_ind = np.max(reg_assign, axis=1) # number of regions assigned to each cuisine

## Rank top 10 words

### c3rLDA

In [127]:
# top10_background = np.argsort(topic_word, axis=1)[:, -20:]

# intersect = np.sum(np(X_train[:, top10_background[0][:2]], axis=1))

# individuals = np.sum(X_train[:, top10_background[0][:2]], axis=0)

# pmi = np.log(10) + np.log(intersect) - np.log(individuals[0, 0]) - np.log(individuals[0, 1])
# npmi = -1 * pmi / (np.log(intersect) - math.log(10))

In [3]:
def calculate_npmi(topics):
    npmi_means = []
    for i, topic in enumerate(topics):
        npmi_vals = []
        words = topic
        print("\rTopic", i, end="")
        for word_i, word1 in enumerate(words):
            for word2 in words[word_i+1:]:
                col1 = np.array(total_X[:, word1].todense() > 0, dtype=int)
                col2 = np.array(total_X[:, word2].todense() > 0, dtype=int)
                c1 = col1.sum()
                c2 = col2.sum()
                c12 = np.sum(col1 * col2)
                if c12 == 0:
                    npmi = 0.0
                else:
                    npmi = (np.log10(total_X.shape[0]) + np.log10(c12) - np.log10(c1) - np.log10(c2)) / (np.log10(total_X.shape[0]) - np.log10(c12))
                npmi_vals.append(npmi)
                #print(str(word1), str(word2))

        npmi_means.append(np.mean(npmi_vals))
        
    print(np.mean(npmi_means))
    return np.mean(npmi_means)

In [4]:
def calculate_npmi_culture(topics, cult):
    npmi_means = []
    for i, topic in enumerate(topics):
        npmi_vals = []
        words = topic
        print("\rTopic", i, end="")
        for word_i, word1 in enumerate(words):
            for word2 in words[word_i+1:]:
                cult_ind = np.where(total_c == cult)[0]
                cult_X = total_X[cult_ind]
                
                col1 = np.array(cult_X[:, word1].todense() > 0, dtype=int)
                col2 = np.array(cult_X[:, word2].todense() > 0, dtype=int)
                c1 = col1.sum()
                c2 = col2.sum()
                c12 = np.sum(col1 * col2)
                print(c1, c2, c12, cult_X.shape[0])
                if c12 == 0:
                    npmi = 0.0
                else:
                    npmi = (np.log10(cult_X.shape[0]) + np.log10(c12) - np.log10(c1) - np.log10(c2)) / (np.log10(cult_X.shape[0]) - np.log10(c12))
                npmi_vals.append(npmi)
                print(npmi)

        npmi_means.append(np.mean(npmi_vals))
    print("\n")
    print(np.mean(npmi_means))
    return np.mean(npmi_means)

In [166]:
top10_background = np.argsort(topic_word, axis=1)[:, -10:]

In [128]:
calculate_npmi(top10_background) # 20

Topic 240.14684326125419578


0.14684326125419578

In [167]:
calculate_npmi(top10_background) # 10

Topic 240.16037791742193427


0.16037791742193427

In [153]:
c3rLDA_nzwc.shape[2]

28

In [150]:
c3rlda_culture_npmi = []

for c in range(c3rLDA_nzwc.shape[2]):
    c3rlda_culture_npmi.append(calculate_npmi_culture(np.argsort(c3rLDA_nzwc[:, :, c], axis=1)[:, -10:], c))
    
np.mean(c3rlda_culture_npmi)

Topic 24

0.05686325874650356
Topic 24

0.08802614708585757
Topic 24

0.05334012141565739
Topic 24

0.18459221277005544
Topic 24

0.08692290028675015
Topic 24

0.07056436710691222
Topic 24

0.060528292515941
Topic 24

0.0932889184570643
Topic 24

0.10582401599915986
Topic 24

0.08595519618782894
Topic 24

0.019278823746995844
Topic 24

0.07182507946082223
Topic 24

0.09967198453238582
Topic 24

0.10525047477477084
Topic 24

0.12960684099845846
Topic 24

-0.0228215457125284
Topic 24

0.03686421017290208
Topic 24

0.07491496459921165
Topic 24

0.13062221238014307
Topic 24

0.11622823789053996
Topic 24

0.08343729229669121
Topic 24

0.03226444595970498
Topic 24

0.09856667078263405
Topic 24

0.006402414239632856
Topic 24

0.10083582460848194
Topic 24

0.0715009214342971
Topic 24

0.11577011157287583
Topic 24

0.12211657614597496


0.08136574894484731

In [161]:
c3rlda_npmi = []

for c in range(c3rLDA_nzwc.shape[2]):
    c3rlda_culture_npmi.append(calculate_npmi(np.argsort(c3rLDA_nzwc[:, :, c], axis=1)[:, -10:]))
    


Topic 240.182273803117267
Topic 240.2421596163887246
Topic 240.3244375789410121
Topic 240.3496355349000767
Topic 240.18360071302318162
Topic 240.23662570678519038
Topic 240.23117198600711336
Topic 240.258262696746809
Topic 240.4436566838337611
Topic 240.3696413614198852
Topic 240.02346801041637384
Topic 240.20513530095738974
Topic 240.2801536318144381
Topic 240.17799091549840168
Topic 240.3636764136274336
Topic 24-0.00909439232484937
Topic 240.17380731351289658
Topic 240.3933446014439055
Topic 240.33994634083766123
Topic 240.29514050190675456
Topic 240.25644693107502803
Topic 240.24251709005523975
Topic 240.3265189050715355
Topic 240.2891807837747459
Topic 240.3169721981691476
Topic 240.2003537984335443
Topic 240.47768746765956804
Topic 240.24588996395957646


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

In [165]:
np.mean(c3rlda_culture_npmi[28:])

0.2650214806089933

# npmi region

In [180]:
region_npmis = []

for c in range(collection_word.shape[2]):
    for r in range(max_reg_ind[c]):
        region_npmis.append(calculate_npmi(np.argsort(collection_word[:, :, c, r], axis=1)[:, -10:]))

Topic 240.22400660121620583
Topic 240.1372402641063569
Topic 240.21791053732718635
Topic 240.24819388547674548
Topic 240.2027595598187921
Topic 240.20192698104515303
Topic 240.2533067048177221
Topic 240.21476594550272718
Topic 240.20829221897272807
Topic 240.23827765380798482
Topic 240.20222579853773187
Topic 240.30096327137279044
Topic 240.35866636797346374
Topic 240.18317799534566184
Topic 240.2046611610637106
Topic 240.3245203719563229
Topic 240.2505879318632263
Topic 240.3054092182746885
Topic 240.12344225524874901
Topic 240.2880759605763383
Topic 240.1455187948696414
Topic 240.30986897344666614
Topic 240.0919758534836212
Topic 240.13274343593567695
Topic 240.15882314989194762
Topic 240.041235642288965355
Topic 240.04645840714240208
Topic 240.18755809307996202
Topic 240.1592860917598987
Topic 240.19133426539057408
Topic 240.20125295712298935
Topic 240.19903625205937783
Topic 240.19724122110338502
Topic 240.19113616723168128
Topic 240.20311431016708842
Topic 240.19058984324280315
To

In [181]:
np.mean(region_npmis)

0.1979319267745276

Marginalized + combined

In [6]:
c3rLDA_nzw = np.sum(c3rLDA_nzwc, axis=2)

In [7]:
c3rLDA_nzw_total = c3rLDA_nzw + topic_word
top10_total = np.argsort(c3rLDA_nzw_total, axis=1)[:, -10:]

In [10]:
calculate_npmi(top10_total) 

Topic 240.15982803835776038


0.15982803835776038

In [11]:
# just marginalized cuisines

calculate_npmi(np.argsort(c3rLDA_nzw, axis=1)[:, -10:]) 

Topic 24-0.00865334163403877


-0.00865334163403877

## LDA

In [142]:
np.where(total_c == 3)[0]

(6474,)

In [98]:
from joblib import dump, load
model = load('./lda/lda.joblib') 

In [107]:
model.components_

array([[1.56558789e+03, 1.58458453e+01, 1.46312687e+00, ...,
        4.00000000e-02, 4.00000000e-02, 4.00000000e-02],
       [8.18525805e+02, 2.33555315e+04, 1.41885869e+01, ...,
        4.00000000e-02, 4.00000000e-02, 4.00000000e-02],
       [1.26333868e+03, 1.00137837e+03, 2.24027260e+02, ...,
        4.00000000e-02, 4.00000000e-02, 4.00000000e-02],
       ...,
       [1.51671417e+03, 1.18636546e+02, 3.50215289e-01, ...,
        4.00000000e-02, 4.00000000e-02, 1.03999998e+00],
       [1.05930723e+04, 9.36455970e-01, 5.72863056e-02, ...,
        4.00000000e-02, 4.00000000e-02, 4.00000000e-02],
       [1.72454466e+03, 1.54000205e+03, 8.44968059e+00, ...,
        4.00000000e-02, 4.00000000e-02, 4.00000000e-02]])

In [168]:
top_10_words= np.argsort(model.components_, axis=1)[:, -10:]

In [132]:
calculate_npmi(top_10_words) # 20

Topic 240.15801556524896346


0.15801556524896346

In [170]:
calculate_npmi(top_10_words) # 10

Topic 240.19354711797315813


0.19354711797315813

## ccLDA

In [109]:
ccLDA_nzw = np.load("./lda/ccLDA_nzw.npy")

In [171]:
top_cc = np.argsort(ccLDA_nzw, axis=1)[:, -10:]

In [126]:
calculate_npmi(top_cc) # 5 .221 - LDA

Topic 240.22433079628165384


0.22433079628165384

In [172]:
calculate_npmi(top_cc) # 10

Topic 240.18779829795701353


0.18779829795701353

In [134]:
calculate_npmi(top_cc) # 20

Topic 240.1668965982455381


0.1668965982455381

In [112]:
ccLDA_nzwc = np.load("./lda/ccLDA_nzwc.npy")

In [179]:
culture_npmi = []

for c in range(ccLDA_nzwc.shape[2]):
    culture_npmi.append(calculate_npmi(np.argsort(collection_word[:, :, c, r], axis=1)[:, -10:]))
    
np.mean(culture_npmi)

KeyboardInterrupt: 

In [151]:
cclda_culture_npmi = []

for c in range(ccLDA_nzwc.shape[2]):
    cclda_culture_npmi.append(calculate_npmi_culture(np.argsort(ccLDA_nzwc[:, :, c], axis=1)[:, -10:], c))
    
np.mean(cclda_culture_npmi)

Topic 24

0.1417480482766072
Topic 24

0.14374050443130007
Topic 24

0.16361669282255323
Topic 24

0.08549038591330332
Topic 24

0.14885100365020212
Topic 24

0.18746340309944837
Topic 24

0.16031614290303098
Topic 24

0.11424504015835261
Topic 24

0.10398422260451959
Topic 24

0.13836743276408198
Topic 24

0.15536100702724162
Topic 24

0.1491425699782948
Topic 24

0.15174319442553785
Topic 24

0.08089752977740894
Topic 24

0.15960367511769977
Topic 24

0.14474223168713535
Topic 24

0.1830209088656038
Topic 24

0.057615820540670895
Topic 24

0.14535879890611164
Topic 24

0.14830451839472295
Topic 24

0.15184544227786323
Topic 24

0.1809745640021049
Topic 24

0.10206797918413227
Topic 24

0.07731787674427057
Topic 24

0.17267616677747955
Topic 24

0.16182074225278378
Topic 24

0.11134587759890174
Topic 24

0.07450638418626117


0.13557743444170084

# try palmetto

In [3]:
from palmettopy.palmetto import Palmetto
palmetto = Palmetto()

from gensim.corpora import Dictionary

unable to import 'smart_open.gcs', disabling that module


In [4]:
our_lexicon = Dictionary.load("/Users/tingchen/Desktop/Yelp Project/big_lexicon_n_20_removed_all_eths_less_than_15000")

inv_lexicon= {v: k for k, v in our_lexicon.token2id.items()}

top10_background = np.argsort(topic_word, axis=1)[:, -10:]

words = np.vectorize(inv_lexicon.get)(top10_background)

NameError: name 'topic_word' is not defined

In [10]:
palmetto_cv = []

for i in range(words.shape[0]):
    palmetto_cv.append(palmetto.get_coherence(words=list(words[i, :].astype(str)), coherence_type="cv"))

In [11]:
np.mean(palmetto_cv)

0.46294999093588024

In [7]:
coherence_measures = ['ca', 'cp', 'npmi', 'umass']

for m in coherence_measures:
    topic_coherence = []
    
    for i in range(words.shape[0]):
        topic_coherence.append(palmetto.get_coherence(words=list(words[i, :].astype(str)), coherence_type=m))
    
    print(m, np.mean(topic_coherence))

ca 0.1626923811865238
cp -0.07756922590512617
npmi -0.011048107168362815
umass -5.535219923648257


Palmetto for Cuisine topics c3rLDA

In [5]:
top10_cuisine = np.argsort(c3rLDA_nzwc, axis=1)[:, -10:, :]

words_cuisine = np.vectorize(inv_lexicon.get)(top10_cuisine)

In [10]:
#coherence_measures = ['cv', 'ca', 'cp', 'npmi', 'umass']
coherence_measures = ['npmi', 'umass']

for m in coherence_measures:
    topic_coherence = []
    
    for c in tqdm(range(c3rLDA_nzwc.shape[2])):
        for i in range(words_cuisine.shape[0]):
            topic_coherence.append(palmetto.get_coherence(words=list(words_cuisine[i, :, c].astype(str)), coherence_type=m))
    
    print(m, np.mean(topic_coherence))

  0%|          | 0/28 [00:00<?, ?it/s]

npmi -0.06200281257102465


100%|██████████| 28/28 [12:28<00:00, 26.74s/it]

umass -7.0001380000842
